# Day 8: Seven Segment Search

[*Advent of Code 2021 day 8*](https://adventofcode.com/2021/day/8) and [*solution megathread*](https://www.reddit.com/rbj87a)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2021/08/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2021%2F08%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

What is this... compute an average?

...

Hmm... nope. It's too early to realize even why that didn't work in Part One, and this solution is so brute force it feels criminal, but it worked!

In [4]:
from IPython.display import display

%pycodestyle_off
testdata = """be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce""".splitlines()
%pycodestyle_on

inputdata = downloaded['input'].splitlines()

In [5]:
display(f'{inputdata[0]}')
display(f'{inputdata[1]}')
display(f'... ({len(inputdata)} lines)')

'fcdeba edcbag decab adcefg acdfb gdcfb acf fabe fa eacfgbd | aefb cfa acf cdabf'

'adbec fabeg fgda gafedb fadeb cdebgf cfaebdg fd bdf cgfbae | ebdfga fbd bdagcef dfb'

'... (200 lines)'

In [6]:
from typing import List, Tuple, Iterable

WiresStr = str


def parse_data(data: List[str]) -> \
        List[Tuple[List[WiresStr], List[WiresStr]]]:
    result = []
    for line in data:
        testpatterns, outputpatterns = line.split(' | ')
        result.append((testpatterns.split(),
                       outputpatterns.split()))
    return result


def my_part1_solution(data: List[str]) -> int:
    count = 0
    for _, outputpatterns in parse_data(data):
        count += sum(len(outputpattern) in (2, 3, 4, 7)
                     for outputpattern in outputpatterns)
    return count

In [7]:
assert my_part1_solution(testdata) == 26

In [8]:
my_part1_solution(inputdata)

449

In [9]:
HTML(downloaded['part1_footer'])

## Part Two

In [10]:
HTML(downloaded['part2'])

Number of segments (not specified which) used in the digits:
```
2 = len(1)
3 = len(7)
4 = len(4)
5 = len(2), len(3), len(5)
6 = len(0), len(6), len(9)
7 = len(8)
```

The digits can be mapped into this table:
```
      a   b   c   d   e   f   g   sum
0     1   1   1   0   1   1   1   6
1     0   0   1   0   0   1   0   2
2     1   0   1   1   1   0   1   5
3     1   0   1   1   0   1   1   5
4     0   1   1   1   0   1   0   4
5     1   1   0   1   0   1   1   5
6     1   1   0   1   1   1   1   6
7     1   0   1   0   0   1   0   3
8     1   1   1   1   1   1   1   7
9     1   1   1   1   0   1   1   6
--
sum   8   6   8   7   4   9   7
```

In [11]:
from enum import Enum


class Digit(Enum):
    ZERO = (True, True, True, False, True, True, True)
    ONE = (False, False, True, False, False, True, False)
    TWO = (True, False, True, True, True, False, True)
    THREE = (True, False, True, True, False, True, True)
    FOUR = (False, True, True, True, False, True, False)
    FIVE = (True, True, False, True, False, True, True)
    SIX = (True, True, False, True, True, True, True)
    SEVEN = (True, False, True, False, False, True, False)
    EIGHT = (True, True, True, True, True, True, True)
    NINE = (True, True, True, True, False, True, True)

    def intvalue(self) -> int:
        for i, d in enumerate(Digit):
            if d == self:
                return i
        raise ValueError(f'unrecognized digit: {self}')

    @staticmethod
    def digits_by_segments() -> Tuple[int, ...]:
        digits = tuple(Digit)
        return tuple(sum(d.value[i] for d in digits)
                     for i in range(len(digits[0].value)))

    @staticmethod
    def segments_by_digits() -> Tuple[int, ...]:
        return tuple(sum(d.value) for d in Digit)

    @staticmethod
    def from_indices(arg_is: List[int]) -> Digit:
        return Digit(tuple(i in arg_is for i in range(7)))

    @staticmethod
    def from_string(arg_str: str) -> Digit:
        return Digit.from_indices([ord(s) - ord('a') for s in arg_str])

    @staticmethod
    def from_int(arg_i: int) -> Tuple[Digit, ...]:
        output = list()
        digits = tuple(Digit)
        if arg_i < 0:
            raise ValueError("only positive integers handled")
        if arg_i == 0:
            return (digits[0], )
        while arg_i > 0:
            d = arg_i % 10
            arg_i //= 10
            output.append(digits[d])
        return tuple(output[::-1])

NameError: name 'Digit' is not defined

In [ ]:
Digit.digits_by_segments()

(8, 6, 8, 7, 4, 9, 7)

In [ ]:
Digit.segments_by_digits()

(6, 2, 5, 5, 4, 5, 6, 3, 7, 6)

In [ ]:
class Wires:
    def __init__(self: Wires, arg_str: str) -> None:
        self.wires = arg_str

    def __str__(self: Wires) -> str:
        return self.wires

    def segment_sum(self: Wires) -> int:
        return len(self.wires)

    @staticmethod
    def sum_over_segments(scrambled_wires: Iterable[Wires]) -> \
            Tuple[Tuple[str, int], ...]:
        output: List[Tuple[str, int]] = []
        for si in range(7):
            s = chr(ord('a') + si)
            output.append((s,
                           sum(s in w.wires
                               for w in scrambled_wires)))
        return tuple(output)

In [ ]:
scrambled_wires = tuple(
    Wires(wires)
    for wires in
    'gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc'.split())
display(f'{tuple(ws.segment_sum() for ws in scrambled_wires)=}')
display(f'{Wires.sum_over_segments(scrambled_wires)=}')

'tuple(ws.segment_sum() for ws in scrambled_wires)=(5, 3, 7, 5, 2, 6, 4, 6, 5, 6)'

"Wires.sum_over_segments(scrambled_wires)=(('a', 7), ('b', 7), ('c', 8), ('d', 4), ('e', 6), ('f', 8), ('g', 9))"

In [ ]:
class TestPattern:
    def __init__(self: TestPattern,
                 scrambled_wires: Tuple[Wires, ...]) -> \
                     None:
        self.scrambled_wires = scrambled_wires

NameError: name 'TestPattern' is not defined

In [ ]:
def my_part2_solution(data: List[str],
                      debug: bool = False) -> int:
    numbers_s = numbers_segments()
    numbers_sum = elems_add(numbers_s.values())
    for testpatterns, outputpatterns in parse_data(data):
        print(f'{testpatterns=} {outputpatterns=}')
    return 0

In [ ]:
my_part2_solution(testdata)

testpatterns=['be', 'cfbegad', 'cbdgef', 'fgaecd', 'cgeb', 'fdcge', 'agebfd', 'fecdb', 'fabcd', 'edb'] outputpatterns=['fdgacbe', 'cefdb', 'cefbgd', 'gcbe']
testpatterns=['edbfga', 'begcd', 'cbg', 'gc', 'gcadebf', 'fbgde', 'acbgfd', 'abcde', 'gfcbed', 'gfec'] outputpatterns=['fcgedb', 'cgb', 'dgebacf', 'gc']
testpatterns=['fgaebd', 'cg', 'bdaec', 'gdafb', 'agbcfd', 'gdcbef', 'bgcad', 'gfac', 'gcb', 'cdgabef'] outputpatterns=['cg', 'cg', 'fdcagb', 'cbg']
testpatterns=['fbegcd', 'cbd', 'adcefb', 'dageb', 'afcb', 'bc', 'aefdc', 'ecdab', 'fgdeca', 'fcdbega'] outputpatterns=['efabcd', 'cedba', 'gadfec', 'cb']
testpatterns=['aecbfdg', 'fbg', 'gf', 'bafeg', 'dbefa', 'fcge', 'gcbea', 'fcaegb', 'dgceab', 'fcbdga'] outputpatterns=['gecf', 'egdcabf', 'bgf', 'bfgea']
testpatterns=['fgeab', 'ca', 'afcebg', 'bdacfeg', 'cfaedg', 'gcfdb', 'baec', 'bfadeg', 'bafgc', 'acf'] outputpatterns=['gebdcfa', 'ecba', 'ca', 'fadegcb']
testpatterns=['dbcfg', 'fgd', 'bdegcaf', 'fgec', 'aegbdf', 'ecdfab', 'fbedc', '

0

In [ ]:
# assert(my_part2_solution(testdata) == 168)

In [ ]:
# my_part2_solution(inputdata)

In [ ]:
HTML(downloaded['part2_footer'])